<a href="https://colab.research.google.com/github/smara97/FakeNews/blob/master/liarplus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from google.colab import files,drive
import collections
import nltk
import torch.nn.functional as F
from nltk.corpus import stopwords
from nltk import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from torch.autograd import Variable
from string import punctuation
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r',encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [0]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1
    emb_matrix = np.zeros((vocab_len,300))
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    return emb_matrix

In [0]:
def clean(text):
  text=text.lower()
  stp=set(stopwords.words("english"))
  placesp = re.compile('[/(){}\[\]\|@,;]')
  removech= re.compile('[^0-9a-z #+_]')
  st=WordNetLemmatizer()
  text=re.sub(placesp,' ',text)
  text=re.sub(removech,' ',text)
  text=text.split()
  text=[w for w in text if not w in stp]
  text=[st.lemmatize(w) for w in text]
  text=" ".join(text)
  text = text.translate(str.maketrans("", "", string.punctuation))
  return text

In [0]:
def transfer_sent(text,word_to_index):
  text=text.split(' ')
  ret=[]
  for w in text:
    if w in word_to_index and w !="":
      ret.append(word_to_index[w])
  return ret

In [0]:
def simlarity(dim1,dim2):
  return (torch.dot(dim1,dim2)/(torch.sqrt(torch.sum(dim1**2))*torch.sqrt(torch.sum(dim2**2))))

In [0]:
def round(pred):
  for i in range(len(pred)):
    if pred[i]>=0.5:
      pred[i]=1
    else:
      pred[i]=0
  return pred

In [0]:
def Accu(pred,labels):
  ret=0
  for i in range(len(labels)):
    if pred[i]==labels[i]:
      ret+=1
  return ret/len(labels)

In [0]:
def convert_to_tensor(training):
  for i in range(len(training)):
    training[i][0]=torch.from_numpy(training[i][0])
    training[i][1]=torch.from_numpy(training[i][1])
    training[i][2]=torch.from_numpy(training[i][2])
    training[i]=torch.cat([training[i][0],training[i][1],training[i][2]])
  return training

In [0]:
def retmax(dftrain):

  stmax,sumax,jumax=0,0,0
  for i in range(dftrain.shape[0]):

    stmax=max(stmax,len(np.array(dftrain.loc[i,'statement'])))

    sumax=max(sumax,len(np.array(dftrain.loc[i,'subject'])))

    jumax=max(jumax,len(np.array(dftrain.loc[i,'justification'])))

  return stmax,sumax,jumax


In [0]:
def convert2D(Xs,max_lens):
   X_indices = np.zeros((Xs[0].shape[0], sum(max_lens)))
   pls=0

   for i in range(Xs[0].shape[0]):
     pls=0
     
     for j in range(0,len(Xs[0][i])):
       X_indices[i][j+pls]=Xs[0][i][j]
     pls=max_lens[0]

     for j in range(0,len(Xs[1][i])):
       X_indices[i][j+pls+1]=Xs[1][i][j]
     pls=max_lens[1]+max_lens[0]

     for j in range(0,len(Xs[2][i])):
       X_indices[i][j+pls+1]=Xs[2][i][j]
   return X_indices

In [0]:
convertlabel = {
	'pants-fire': 0,
	'false': 0,
	'barely-true': 0,
	'half-true': 1,
	'mostly-true': 1,
	'true': 1
}

In [0]:
cols=['index','ID','label','statement','subject','speaker',
      'speaker_job','state','party','barely_true',
      'false','half_true','mostly_true','pants_on_fire',
      'context','justification']

In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs("/content/drive/My Drive/Datasets/Word Embedding/glove.6B.300d.txt")
word_embedding=pretrained_embedding_layer(word_to_vec_map, word_to_index)


In [0]:
dftrain=pd.read_csv("/content/drive/My Drive/Datasets/liar-plus/train.tsv",sep="\t",header=None)
dfval=pd.read_csv("/content/drive/My Drive/Datasets/liar-plus/val.tsv",sep="\t",header=None)
dftest=pd.read_csv("/content/drive/My Drive/Datasets/liar-plus/test.tsv",sep="\t",header=None)

In [0]:
dftrain.columns=cols
dfval.columns=cols
dftest.columns=cols

In [0]:
dftrain=dftrain.loc[:,['statement','subject','justification','label']]
dfval=dfval.loc[:,['statement','subject','justification','label']]
dftest=dftest.loc[:,['statement','subject','justification','label']]
dftrain=dftrain.append(dfval)
dftrain=dftrain.append(dftest)

In [0]:
dftrain=dftrain.dropna(axis=0)

dftrain=dftrain.reset_index()

dftrain=dftrain.drop(['index'],axis=1)


In [20]:
dftrain.head(2)

,statement,subject,justification,label
0,Says the Annies List political group supports ...,abortion,That's a premise that he fails to back up. Ann...,false
1,When did the decline of coal start? It started...,"energy,history,job-accomplishments","Surovell said the decline of coal ""started whe...",half-true


In [0]:
for i in range(dftrain.shape[0]):
  dftrain.loc[i,'label']=convertlabel[dftrain.loc[i,'label']]
  dftrain.loc[i,'statement']=transfer_sent(clean(dftrain.loc[i,'statement']),word_to_index)
  dftrain.loc[i,'subject']=transfer_sent(clean(dftrain.loc[i,'subject']),word_to_index)
  dftrain.loc[i,'justification']=transfer_sent(clean(dftrain.loc[i,'justification']),word_to_index)

In [0]:
dftrainy=dftrain['label']

dftrain=dftrain.drop(['label'],axis=1)

In [0]:
stmax,sumax,jumax=retmax(dftrain)
Fulldata=np.array(convert2D([dftrain.statement,dftrain.subject,dftrain.justification],[410,30,1000]))


In [0]:
dftrainy=list(dftrainy)
training,trainingy=Fulldata[:10153],dftrainy[:10153]
validation,validationy=Fulldata[10154:11422],dftrainy[10154:11422]
testing,testingy=Fulldata[11423:],dftrainy[11423:]

In [0]:
training=torch.from_numpy(training)
trainingy = torch.tensor(trainingy) 
train_tensor = torch.utils.data.TensorDataset(training, trainingy)

validation=torch.from_numpy(validation)
validationy = torch.tensor(validationy) 
valid_tensor = torch.utils.data.TensorDataset(validation, validationy)

testing=torch.from_numpy(testing)
testingy = torch.tensor(testingy) 
test_tensor = torch.utils.data.TensorDataset(testing, testingy)

In [0]:
train_loader=torch.utils.data.DataLoader(dataset=train_tensor,batch_size=64,shuffle=True, num_workers=0)
vali_loader=torch.utils.data.DataLoader(dataset=valid_tensor,batch_size=64,shuffle=True, num_workers=0)
test_loader=torch.utils.data.DataLoader(dataset=test_tensor,batch_size=64,shuffle=True, num_workers=0)


In [0]:
class NN(nn.Module):


  def __init__(self, vocab_size, embedding_dim):

    super(NN, self).__init__()

        
    self.embedding = nn.Embedding(vocab_size, embedding_dim)   

    self.fc = nn.Linear(903, 256)
    self.relu1=nn.ReLU()
    self.dropout1=nn.Dropout(0.4)

    self.fc2=nn.Linear(256,1)
    self.sig = nn.Sigmoid()
        

  def forward(self, x):

    batch_size = x.size(0)

    x = self.conv(x)

    out = self.dropout1(self.relu1(self.fc(x)))
    out=self.sig(self.fc2(out))
    return out
    
  def conv(self,x):
    batch=len(x)
    ret=torch.zeros((batch,903)).cuda()
    st=torch.zeros(300).cuda()
    su=torch.zeros(300).cuda()
    ju=torch.zeros(300).cuda()

    for i in range(len(x)):
      cnt=0
      for j in range(0,411):
        if (x[i][j]!=0):
          cnt+=1
          st+=self.embedding(x[i][j])
      st/=cnt
      cnt=0
      for j in range(411,441):
        if (x[i][j]!=0):
          cnt+=1
          su+=self.embedding(x[i][j])
      su/=cnt

      cnt=0
      for j in range(411,1440):
        if (x[i][j]!=0):
          cnt+=1
          ju+=self.embedding(x[i][j])
      ju/=cnt

      indx=0
      for j in range(len(st)):
        ret[i][indx]=st[j]
        indx+=1
      ret[i][indx]=simlarity(st,su)
      indx+=1
      for j in range(len(su)):
        ret[i][indx]=su[j]
        indx+=1

      ret[i][indx]=simlarity(su,ju)
      indx+=1

      for j in range(len(ju)):
        ret[i][indx]=ju[j]
        indx+=1

      ret[i][indx]=simlarity(st,ju)
    return ret
      

In [37]:
vocab_size = len(word_to_index)+1
embedding_dim = 300
net = NN(vocab_size, embedding_dim)
print(net)

NN(
  (embedding): Embedding(400001, 300)
  (fc): Linear(in_features=903, out_features=256, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [0]:
lr=0.01

net.cuda()
criterion = nn.BCELoss()

optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [0]:
epochs = 4
net.train()
for e in range(epochs):
  Taccuracy,Vaccuracy=[],[]
  losses=[]
  
  for inputs, labels in train_loader:

    inputs, labels = inputs.cuda(), labels.cuda()

   
    net.zero_grad()
    
    output= net(inputs.long())
    Taccuracy.append(Accu(torch.round(output),labels))
    torch.autograd.set_detect_anomaly(True)

    loss = criterion(output.squeeze().float(), labels.float())
    loss = Variable(loss, requires_grad = True)
    losses.append(loss.item())
    loss.backward()

    optimizer.step()
  print("Epoch: {}/{}...".format(e+1, epochs),"Training Finished")
  with torch.no_grad():

    net.eval()
    val_losses = []
  
    for inputs, labels in vali_loader:

      inputs, labels = inputs.cuda(), labels.cuda()

      output= net(inputs.long())
      Vaccuracy.append(Accu(torch.round(output),labels))
      val_loss = criterion(output.squeeze(), labels.float())

      val_losses.append(val_loss.item())

    net.train()

    print("Epoch: {}/{}...".format(e+1, epochs),
          "Tarining Loss: {:.6f}...".format(np.mean(losses)),
          "Val Loss: {:.6f}".format(np.mean(val_losses)),
          "Val Accu:{:.6f}".format(np.mean(Vaccuracy)),
          "Training Accu:{:.6f}".format(np.mean(Taccuracy)))
    

In [0]:
ex,lb=next(iter(vali_loader))
net.eval()
out=net(ex.cuda().long())

In [42]:
print(out.squeeze())

tensor([0.5246, 0.5155, 0.5633, 0.5441, 0.5258, 0.5108, 0.5207, 0.5465, 0.5531,
        0.5472, 0.5256, 0.5301, 0.5138, 0.5189, 0.5227, 0.5230, 0.5166, 0.4974,
        0.5391, 0.5183, 0.5123, 0.5129, 0.5245, 0.5396, 0.5468, 0.5251, 0.5352,
        0.5359, 0.5259, 0.4777, 0.5189, 0.4743], device='cuda:0',
       grad_fn=<SqueezeBackward0>)


In [0]:
loo=criterion(out.squeeze(),lb.cuda().float())

In [45]:
loo.item()

0.6827772259712219